In [1]:
import nltk
import pandas as pd
import wordcloud
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import re
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from keras.utils import np_utils
from collections import defaultdict
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from nltk.tokenize import RegexpTokenizer
from sklearn import metrics

In [2]:
data = pd.read_csv('Dataset.csv')
data.head()

,Unnamed: 0,studentAnswer,referenceAnswer,ref_1,ref_2,accuracy
0,0,By letting it sit in a dish for a day.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect
1,1,Let the water evaporate and the salt is left b...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,correct
2,2,The water evaporated and left salt crystals.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,correct
3,3,I saw a pinkish grayish color that was blockin...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect
4,4,You have to slowly tip the vial for only the w...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect


In [3]:
label_encoder = preprocessing.LabelEncoder()
data ['accuracy'] = label_encoder.fit_transform(data['accuracy'])
student_answer = data['studentAnswer']
ref_answers = data['referenceAnswer']
ref_1= data['ref_1']
ref_2= data['ref_2']
ps = PorterStemmer()
stop_words = set(stopwords.words("english"))

In [4]:
def clean(s):
  out = word_tokenize(s)
  ans = ' '.join([''.join(ps.stem(word)) for word in out if word not in stop_words])
  return ans
filtered_answers = [clean(s) for s in student_answer]

In [5]:
print(filtered_answers[0])
print(student_answer[0])

By let sit dish day .
By letting it sit in a dish for a day.


In [6]:
def clean(s):
  out = word_tokenize(s)
  ans = ' '.join([''.join(ps.stem(word)) for word in out if word not in stop_words])
  return ans
filtered_answers_ref2 = [clean(s) for s in ref_2]

In [7]:
print(filtered_answers_ref2[0])
print(ref_2[0])

I put water dish let water evapor salt would stay .
I put some water in a dish and let the water evaporate so the salt would stay.


In [8]:
answers= filtered_answers+filtered_answers_ref2

In [9]:
l=len(filtered_answers)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(data['studentAnswer'], data['accuracy'], random_state=0,test_size = 0.25)

In [11]:
print(data['accuracy'].value_counts())

1    2944
0    1981
Name: accuracy, dtype: int64


In [12]:
print("x_train shape: {}".format(x_train.shape), end='\n')
print("y_train shape: {}".format(y_train.shape), end='\n\n')
print("x_test shape: {}".format(x_test.shape), end='\n')
print("y_test shape: {}".format(y_test.shape), end='\n\n')

x_train shape: (3693,)
y_train shape: (3693,)

x_test shape: (1232,)
y_test shape: (1232,)



In [13]:
x_train

2864    I think she made a solution because the eggshe...
685     Overly Orange and Luscious Lemon have more vit...
2260    Maybe Greta has a bigger hand and Andre smalle...
3935    The blue covered got hotter and in the same am...
221                  The salt crystals look like squares.
                              ...                        
4859    They need to have thrown it at the same place ...
3264                           It will make a doom sound.
1653    C because it is on the square part and on the ...
2607    The alum has the same pattern and it looks the...
2732    Because they are both at the highest point of ...
Name: studentAnswer, Length: 3693, dtype: object

In [14]:
X_train = answers
# print(test_sent)
#tfidf = TfidfVectorizer()
#tfidf = tfidf.fit_transform(x_train)
tfidf = TfidfVectorizer(min_df=5) #minimum document frequency of 5
X_train_tfidf = tfidf.fit_transform(x_train)
students_answers = X_train_tfidf[:l]
model_answers = X_train_tfidf[l:]


print("Number of features : %d \n" %len(tfidf.get_feature_names())) #1722
print("Show some feature names : \n", tfidf.get_feature_names()[::1000])

Number of features : 824 

Show some feature names : 
 ['10']


In [15]:
# naive bayes
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
for x in range(20):
    print("naive bayes comparing student answer" , x, "and model answer",x , "is" ,MultinomialNB(students_answers[x], model_answers[x]))


ValueError: Expected 2D array, got 1D array instead:
array=['By let sit dish day .' 'let water evapor salt left behind .'
 'the water evapor left salt crystal .' ...
 'you use standard tell happen control experi chang one variabl time .'
 'you use standard tell happen control experi chang one variabl time .'
 'you use standard tell happen control experi chang one variabl time .'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
X_test = tfidf.fit_transform(x_test)

In [ ]:
# Predicting the Test set results  
y_pred =mnb.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
ConfMat=confusion_matrix(y_test, y_pred)
print(ConfMat)

In [ ]:
import seaborn as sns
ax = sns.heatmap(ConfMat/np.sum(ConfMat),fmt='.2%', annot=True, cmap='Blues')

ax.set_title(' Confusion Matrix \n\n');
ax.set_xlabel('\n Predicted')
ax.set_ylabel('Actual ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['1', '2'])
ax.yaxis.set_ticklabels(['1', '2',])

## Display the visualization of the Confusion Matrix.
##plt.show()